In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


잘못해서 삭제된 라벨링 파일 다시 생성

In [ ]:
import os
import json
import cv2

# 경로 설정
txt_folder = 'D:/JOLJAK/TOMATO_Train'
json_folder = 'D:/JOLJAK/e.열매'

# 바운딩 박스 좌표 계산 함수
def polygon_to_bbox(polygon):
    x_coordinates = [point[0] for point in polygon]
    y_coordinates = [point[1] for point in polygon]
    x_min = min(x_coordinates)
    x_max = max(x_coordinates)
    y_min = min(y_coordinates)
    y_max = max(y_coordinates)
    return x_min, y_min, x_max, y_max

# 용량이 0KB인 txt 파일을 찾고, 해당 json 파일에서 바운딩 박스 좌표를 추출하여 txt 파일에 저장
for txt_file in os.listdir(txt_folder):
    txt_path = os.path.join(txt_folder, txt_file)
    
    # 용량이 0KB인 txt 파일 확인
    if os.path.isfile(txt_path) and os.path.getsize(txt_path) == 0:
        json_file = txt_file.replace('.txt', '.json')
        json_path = os.path.join(json_folder, json_file)
        
        # json 파일이 존재하는지 확인
        if os.path.exists(json_path):
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # 'shapes' 키에 폴리곤 정보가 있다고 가정
            if 'shapes' in data:
                with open(txt_path, 'w') as txt:
                    for shape in data['shapes']:
                        polygon = shape['points']  # 폴리곤 좌표
                        class_id = shape.get('label', 0)  # 클래스 ID (또는 기본값 0)
                        
                        # 폴리곤을 바운딩 박스로 변환
                        x_min, y_min, x_max, y_max = polygon_to_bbox(polygon)
                        
                        # 바운딩 박스를 YOLO 형식으로 변환 (중심 좌표와 너비, 높이 비율로 변환)
                        img_width = data['imageWidth']
                        img_height = data['imageHeight']
                        cx = (x_min + x_max) / 2 / img_width
                        cy = (y_min + y_max) / 2 / img_height
                        width = (x_max - x_min) / img_width
                        height = (y_max - y_min) / img_height
                        
                        # YOLO 형식으로 txt 파일에 저장
                        txt.write(f"{class_id} {cx} {cy} {width} {height}\n")
                    print(f"Updated {txt_path} with bounding box data from {json_path}")
        else:
            print(f"JSON file not found for {txt_file}")


Updated D:/JOLJAK/TOMATO_Train\V001_tom1_00_061_e_14_20220101_27_05094434_40158887.txt with bounding box data from D:/JOLJAK/e.열매\V001_tom1_00_061_e_14_20220101_27_05094434_40158887.json
Updated D:/JOLJAK/TOMATO_Train\V001_tom1_00_062_e_14_20220101_27_05094518_40158887.txt with bounding box data from D:/JOLJAK/e.열매\V001_tom1_00_062_e_14_20220101_27_05094518_40158887.json
Updated D:/JOLJAK/TOMATO_Train\V001_tom1_00_062_e_15_20220101_27_05094550_40158887.txt with bounding box data from D:/JOLJAK/e.열매\V001_tom1_00_062_e_15_20220101_27_05094550_40158887.json
Updated D:/JOLJAK/TOMATO_Train\V001_tom1_00_062_e_15_20220101_27_06094550_40158887.txt with bounding box data from D:/JOLJAK/e.열매\V001_tom1_00_062_e_15_20220101_27_06094550_40158887.json
Updated D:/JOLJAK/TOMATO_Train\V001_tom1_00_063_e_14_20220101_27_05094942_40158887.txt with bounding box data from D:/JOLJAK/e.열매\V001_tom1_00_063_e_14_20220101_27_05094942_40158887.json
Updated D:/JOLJAK/TOMATO_Train\V001_tom1_00_064_e_14_20220101_27_

라벨 추출

In [2]:
import json
import os

def extract_labels_from_json(json_file):
    """
    JSON 파일에서 라벨 정보를 추출하여 리스트로 반환합니다.
    
    Parameters:
    - json_file (str): JSON 파일의 경로.
    
    Returns:
    - labels (set): 중복 제거된 라벨들의 집합.
    """
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    shapes = data.get('shapes', [])
    labels = {shape['label'] for shape in shapes if 'label' in shape}
    
    return labels

def process_folders(train_labels):
    all_labels = set()  # 중복 제거를 위한 집합 사용
    
    # 각 폴더를 순회하며 JSON 파일에서 라벨을 추출
    for folder in train_labels:
        if os.path.exists(folder):
            json_files = [f for f in os.listdir(folder) if f.endswith('.json')]
            
            for json_file in json_files:
                json_path = os.path.join(folder, json_file)
                labels = extract_labels_from_json(json_path)
                all_labels.update(labels)
        else:
            print(f"Folder not found: {folder}")
    
    return sorted(list(all_labels))

# 폴더 리스트
train_labels = [
    "C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터/c.열매",
    "C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터/a3.줄기두께",
    "C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터/f.만개꽃",
    "C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터/b.꽃",
    "C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터/a1.생장길이",
    "C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터/a2.최상위개화꽃높이",
    "C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터/a4.엽장엽폭"
]

# 모든 폴더에서 라벨 추출
class_labels = process_folders(train_labels)

# 추출된 클래스 라벨 출력
print(f"추출된 클래스 라벨: {class_labels}")


추출된 클래스 라벨: ['pap_c_flower_poly', 'pap_flower_center_bb', 'pap_flower_poly', 'pap_flower_stem_bb', 'pap_flower_tip_bb', 'pap_fruit_color_poly', 'pap_fruit_green_poly', 'pap_growth_bb', 'pap_leaf_bb', 'pap_p_flower_poly', 'pap_stem_dimeter_bb', 'pap_stem_dimeter_tilt_bb']


JSON YOLO 형식으로

In [ ]:
import os
import json

# 경로 설정
input_folder = 'C:/Users/ahsld/Downloads/098.지능형 스마트팜 통합 데이터(파프리카)/01.데이터/1.Training/라벨링데이터'
output_folder = 'C:/Users/ahsld/Desktop/PAPRIKA/PAP_Train'
os.makedirs(output_folder, exist_ok=True)

# YOLO 클래스 ID 매핑
class_mapping = {
    "pap_c_flower_poly": 0,
    "pap_flower_center_bb": 1,
    "pap_flower_poly": 2,
    "pap_flower_stem_bb": 3,
    "pap_flower_tip_bb": 4,
    "pap_fruit_color_poly": 5,
    "pap_fruit_green_poly": 6,
    "pap_growth_bb": 7,
    "pap_leaf_bb": 8,
    "pap_p_flower_poly": 9,
    "pap_stem_dimeter_bb": 10,
    "pap_stem_dimeter_tilt_bb": 11
}

# 하위 폴더 탐색 및 JSON 파일 처리
for root, _, files in os.walk(input_folder):
    for file_name in files:
        if file_name.endswith('.json'):
            input_path = os.path.join(root, file_name)
            
            # JSON 파일 읽기
            with open(input_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # YOLO 라벨 파일 생성
            yolo_label = []
            img_width, img_height = data["imageWidth"], data["imageHeight"]

            for shape in data["shapes"]:
                label = shape["label"]
                if label in class_mapping:
                    points = shape["points"]
                    x_min = min([p[0] for p in points])
                    y_min = min([p[1] for p in points])
                    x_max = max([p[0] for p in points])
                    y_max = max([p[1] for p in points])

                    # YOLO 형식으로 좌표 변환
                    bbox_center_x = (x_min + x_max) / 2 / img_width
                    bbox_center_y = (y_min + y_max) / 2 / img_height
                    bbox_width = (x_max - x_min) / img_width
                    bbox_height = (y_max - y_min) / img_height

                    # 라벨 문자열 생성 (클래스, x중심, y중심, 너비, 높이)
                    yolo_label.append(f"{class_mapping[label]} {bbox_center_x} {bbox_center_y} {bbox_width} {bbox_height}")

            # YOLO 형식의 라벨 파일 저장 (출력 경로에 파일 이름만)
            output_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}.txt")
            with open(output_path, 'w') as f:
                f.write("\n".join(yolo_label))

print("YOLO 라벨링 파일 변환 완료!")


KeyboardInterrupt: 

In [4]:
import os
import glob

# 경로 설정
folder_path = 'D:/JOLJAK/TOMATO_Val'

# 폴더 내 모든 .txt 파일 찾기
txt_files = glob.glob(os.path.join(folder_path, '*.txt'))

# 모든 .txt 파일 삭제
for txt_file in txt_files:
    os.remove(txt_file)

print("모든 .txt 파일이 삭제되었습니다.")


모든 .txt 파일이 삭제되었습니다.
